#1. SPD Matrices and $LDL^t$ Factorization

Given an SPD matrix $A$, it can be factorized into $LDL^T$, where $L$ is an unitary lower triangular matrix and $D$ is a diagonal matrix.  To construct the factorization, we follow the Algorithm 6.5 on page 417:
* Do i = 1, ..., n
  * (Step2): For $j = 1,\dots, i-1$, set $v_j = l_{ij}d_j$ 
  * (Step3): Set $d_i = a_{ii} - \displaystyle\sum_{j=1}^{i-1} l_{ij}v_j$ 
  * (Step4): For $j = i+1, \dots, n$, set $l_{ji} = \big( a_{ji} - \displaystyle\sum_{k=1}^{i-1}l_{jk}v_k \big) /d_i$

In [ ]:
function LDLt(A)
    # Given an SPD matrix A_{n, n}
    # Output L and D
    n, n1 = size(A)
    v = zeros(n,1)
    L = eye(n)
    d = zeros(n,1)
    for i = 1:n
        # step 2
        for j = 1:i-1
            v[j] = L[i,j] * d[j]
        end
        
        # step 3
        temp = 0
        for j = 1:i-1
            temp = temp + L[i,j] * v[j]
        end
        d[i] = A[i,i] - temp
        
        #step 4
        for j = i+1:n
            temp = 0
            for k = 1:i-1
                temp = temp + L[j,k] * v[k]
            end
            L[j,i] = (A[j,i] - temp) / d[i]
        end
    end
    return L, d
end

Test the code on Example 3 on page 418:
$$A =
\begin{pmatrix}
4 & -1 & 1 \\
-1 & 4.25 & 2.75 \\
1 & 2.75 & 3.5
\end{pmatrix}
,
$$
and its $LDL^t$ factorization is
$$L = 
\begin{pmatrix}
1 & 0 & 0 \\
-0.25 & 1 & 0 \\
0.25 & 0.75 & 1
\end{pmatrix}
\quad
D = 
\begin{pmatrix}
4 & 0 & 0 \\
0 & 4 & 0 \\
0 & 0 & 1
\end{pmatrix}
.
$$

In [ ]:
A = [4 -1 1; -1 4.25 2.75; 1 2.75 3.5]
LDLt(A)

#2. Tridiagonal Matrix
Tridiagonal matrix can be factorized into $LU$, where $L$ is a lower triangular matrix, $U$ is an unitary upper triangular matrix, and both of them have tridiagonal form.
* $l_{11} = a_{11}$
* $u_{12} = a_{12}/l_{11}$
* Do i = 2, ..., n-1
  * $l_{i, i-1} = a_{i,i-1}$
  * $l_{ii} = a_{ii} - l_{i,i-1} u_{i-1, i}$
  * $u_{i,i+1} = a_{i, i+1}/l_{ii}$
* $l_{n,n-1} = a_{n,n-1}$
* $l_{n,n} = a_{nn} - l_{n,n-1} u_{n-1,n}$

In [ ]:
function tridiagLU(A)
    n, n1 = size(A)
    L = zeros(n,n)
    U = eye(n)
    L[1,1] = A[1,1]
    U[1,2] = A[1,2] / L[1,1]
    for i = 2:n-1
        L[i, i-1] = A[i,i-1]
        L[i,i] = A[i,i] - L[i,i-1] * U[i-1,i]
        U[i,i+1] = A[i,i+1] / L[i,i]
    end
    L[n,n-1] = A[n,n-1]
    L[n,n] =  A[n,n] - L[n,n-1]* U[n-1,n]
    return L, U
end

Test it on Example 5 on page 423:
$$
T = 
\begin{pmatrix}
2 & -1 & 0 & 0 \\
-1 & 2 & -1& 0 \\
0 & -1 & 2 & -1 \\
0 & 0 & -1 & 2
\end{pmatrix}
$$
and its LU factorization is 
$$
L = 
\begin{pmatrix}
2 & 0 & 0 & 0 \\
-1 & \frac{3}{2} & 0 & 0 \\
0 & -1 & \frac{4}{3} & 0 \\
0 & 0 & -1 & \frac{5}{4}
\end{pmatrix}
,\quad
U = 
\begin{pmatrix}
1 & -\frac{1}{2} & 0 & 0 \\
0 & 1 & -\frac{2}{3} & 0 \\
0 & 0 & 1 & -\frac{3}{4} \\
0 & 0 & 0 & 1
\end{pmatrix}.
$$

In [ ]:
T = [2 -1 0 0; -1 2 -1 0; 0 -1 2 -1; 0 0 -1 2]
tridiagLU(T)

#3. Jacobi's Method
The Jacobi's iterative method is to find the solution of $Ax=b$, and can be written as for given $x^{(0)}$,
$$x^{(k)}_i = \frac{1}{a_{ii}} \big[ \sum_{j=1,\;j\ne i}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big]$$, or equivalently
$$x^{(k)}_i = \frac{1}{a_{ii}} \big[ \sum_{j=1}^{i-1} (-a_{ij}x^{(k-1)}_{j}) + \sum_{j=i+1}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big].$$

> Note that super-script, k, represents the iteration numbers, and sub-script, i, represents the i-th entry of x.

In [ ]:
function jacobi(A,b, x0, TOL, Nmax)
    n, n1 = size(A)
    x = zeros(n,1)
    for k = 1:Nmax
        for i = 1:n
            temp1 = 0
            for j = 1:i-1
                temp1 = temp1 + A[i,j] * x0[j]
            end
            
            temp2 = 0
            for j = i+1:n
                temp2 = temp2 + A[i,j] * x0[j]
            end
            
            x[i] = (-temp1 - temp2 + b[i]) /  A[i,i]
        end
        res = norm(x - x0, Inf)
        #println(k, '\t', res)
        if res < TOL
            return x
        end
        x0[1:n] = x[1:n]
    end
    println("Reach the maximum iterations! Solution might be wrong")
end

Test it on Example 1 on page 451.  The linear system $Ax = b$ is given by
$$
\begin{cases}
 &10x_1 - x_2 + 2x_3 &= 6,\\
 &-x_1 + 11x_2 - x_3 + 3x_4 &= 25,\\
 &2x_1 - x_2 + 10x_3 - x_4 &= -11,\\
 & 3x_2 - x_3 + 8x_4 &=15
\end{cases}
,
$$
and the exact solution is $(1, 2, -1, 1)$.

In [ ]:
A1 = [10 -1 2 0; -1 11 -1 3; 2 -1 10 -1; 0 3 -1 8]
b = [6;25;-11;15]
jacobi(A1, b, zeros(4,1), 1e-8, 100)

#4. Gauss-Seidel Method
The GS method can be written as
$$x^{(k)} = \frac{1}{a_{ii}} \big[ \sum_{j=1}^{i-1} (-a_{ij}x^{(k)}_{j}) + \sum_{j=i+1}^{n} (-a_{ij}x^{(k-1)}_{j}) + b_i \big].$$

In [ ]:
function GS(A, b, x0, TOL, Nmax)
    n, n1 = size(A)
    x = zeros(n,1)
    for k = 1:Nmax
        for i = 1:n
            temp1 = 0
            for j = 1:i-1
                temp1 = temp1 + A[i,j] * x[j]
            end
            
            temp2 = 0
            for j = i+1:n
                temp2 = temp2 + A[i,j] * x0[j]
            end
            
            x[i] = ((-temp1) - temp2 + b[i]) /  A[i,i]
        end
        res = norm(x - x0, Inf) # make sure calculating inf norm
        #println(k, '\t', res)
        if res < TOL
            return x
        end
        x0[1:n] = x[1:n]
    end
    println("Reach the maximum iterations! Solution might be wrong")
end

In [ ]:
GS(A1, b, zeros(4,1), 1e-8, 100)

In [ ]:
# 7.3.9b
# Show that the Jacobi method with x^(0) = 0 fails to give a good approximation after 25 iterations.
A_9 = [
    2 -1 1;
    2 2 2;
    -1 -1 2
]
b_9 = [-1; 4; -5]
x0 = zeros(3,1)
TOL = 0.01
Nmax = 25
x_j = jacobi(A_9, b_9, x0, TOL, Nmax)
println(x_j)
x_gs = GS(A_9, b_9, x0, TOL, Nmax)
println(x_gs)

Residual is increasing

In [ ]:
# 7.3.9d
A = [2 -1 1; 2 2 2; -1 -1 2]
b = [-1; 4; -5]
x0 = zeros(3,1)
TOL = 1e-5
Nmax = 100
x2 = GS(A, b, x0, TOL, Nmax)

In [126]:
# 7.3.15
A_15 = zeros(80,80)
for i = 1:80
    A_15[i,i] = 2*i
    if i + 2 <= 80
        A_15[i, i+2] = 0.5*i
    end
    if i - 2 >= 1
        A_15[i, i-2] = 0.5*i
    end
    if i + 4 <= 80
        A_15[i, i+4] = 0.25*i
    end
    if i - 4 >= 1
        A_15[i, i-4] = 0.25*i
    end
end
b_15 = zeros(80,1)
for i = 1:80
    b_15[i] = pi
end

In [127]:
# 7.3.15 Jacobi
TOL = 1e-5
Nmax = 100
x0 = zeros(80,1)
jacobi(A_15, b_15, x0, TOL, Nmax)

1	

80x1 Array{Float64,2}:
 1.53874  
 0.731422 
 0.107971 
 0.173285 
 0.0405586
 0.0852502
 0.16645  
 0.121982 
 0.101253 
 0.0904597
 0.0720317
 0.070266 
 0.0687583
 ⋮        
 0.0129717
 0.0127866
 0.0127033
 0.0125272
 0.012377 
 0.0122101
 0.0112904
 0.0111414
 0.0121734
 0.0120177
 0.0154291
 0.0152381

1.5707963267948966
2	0.49928883244552064
3	0.21573330962424286
4	0.13580071609269906
5	0.08927768748178311
6	0.05738343260429862
7	0.03956042258643813
8	0.027304895862305986
9	0.019013610337710943
10	0.013354587126020737
11	0.009432115988693784
12	0.006738333732502405
13	0.004817905298950667
14	0.003448958147223069
15	0.0024818149592938515
16	0.0017963314611503245
17	0.001300812439306992
18	0.0009424946643308008
19	0.000683323389126772
20	0.0004982848643239057
21	0.0003637541075812642
22	0.0002656164153766738
23	0.00019401764465539623
24	0.00014176979919040128
25	0.00010392978668904984
26	7.630210753155342e-5
27	5.602709918651333e-5
28	4.114700333039678e-5
29	3.0225052185944845e-5
30	2.2207134839594134e-5
31	1.636493584597498e-5
32	1.206391119972644e-5
33	8.894144361740186e-6


In [ ]:
# 7.3.15 Gauss-Seidel
TOL = 1e-5
Nmax = 100
x0 = zeros(80,1)
GS(A_15, b_15, x0, TOL, Nmax)

In [ ]:
# 7.3.16b
n = 100
A = eye(n)
for i = 1:n
    if i != 1
        A[i,i-1] = -1/2
    end
    if i != n
        A[i, i+1] = -1/2
    end
end
A

b = zeros(n,1)
b[1] = 1/2

In [ ]:
# Gauss-Sidel
x0 = zeros(n,1)
TOL = 1e-8
Nmax = 20000
GS(A, b, x0, TOL, Nmax)

In [ ]:
# LDLt
L, d = LDLt([A b])
D = zeros(n,n)
for i = 1:n
    D[i,i] = d[i]
end
y = L\b
z = D\y
x = transpose(L)\z
println(x)

In [ ]:
# Tridiagonal LU decomposition
L, U = tridiagLU([A b])
y = L\b
x = U\y
println(x)